In [2]:
import pickle
import math
import time
import numpy as np
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

In [65]:
temp = np.arange(1, 800 * 32 + 1)
temp = temp.reshape(-1, 10)
temp_data = []
for i in range(temp.shape[0]):
    temp_data.append([temp[i].tolist()])
data = {}
data['user'] = temp_data

In [69]:
# with open('data/filtered_data.pkl', 'rb') as f:
#     data = pickle.load(f)

arr = []
sessions = 0
for user in data:
    temp = data[user]
    sessions += len(temp)
    for lis in temp:
        arr += lis[0]
        
print("Users: ", len(data))
print("Sessions: ", sessions)
songs = set(arr)
print("Songs: ", len(songs))
print("Logs: ", len(arr))

song_to_ind = {}
ind_to_song = {}
vocab_sz = 0
for song in songs:
    song_to_ind[song] = vocab_sz
    ind_to_song[vocab_sz] = song
    vocab_sz += 1
extra_tokens = ['<pad>']
for token in extra_tokens:
    song_to_ind[token] = vocab_sz
    ind_to_song[vocab_sz] = token
    vocab_sz += 1
vocab_sz = len(song_to_ind)

sentences = []
for user in data:
    for session in data[user]:
        if (len(session[0]) <= 20):
            sentences += session
train_len = int(len(sentences) * 0.70)
train_data = sentences[:train_len]
test_data = sentences[train_len:]
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        sentences[i][j] = song_to_ind[sentences[i][j]]

Users:  1
Sessions:  2560
Songs:  25600
Logs:  25600


In [71]:
BATCH_SIZE = 32
PAD_IDX = song_to_ind['<pad>']
# BOS_IDX = song_to_ind['<bos>']
# EOS_IDX = song_to_ind['<eos>']

def generate_batch(data_batch):
    batch = []
    target = []
    for item in data_batch:
        total_len = len(item)
#         batch.append(torch.cat([torch.tensor([BOS_IDX]), torch.tensor(item[:total_len - 1]), torch.tensor([EOS_IDX])], dim=0))
        batch.append(torch.tensor(item[:total_len - 1]))
        target.append(item[total_len - 1])
    batch = pad_sequence(batch, padding_value=PAD_IDX)
    target = torch.tensor(target).unsqueeze(0)
    return batch, target

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [233]:
# class Encoder(nn.Module):
#     def __init__(self, input_size, embed_size, hidden_size, n_layers=1, dropout=0):
#         super(Encoder, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.embed_size = embed_size
#         self.embed = nn.Embedding(input_size, embed_size)
#         self.gru = nn.GRU(embed_size, hidden_size, n_layers, dropout=dropout, bidirectional=True)

#     def forward(self, src, hidden=None):
#         embedded = self.embed(src)
#         outputs, hidden = self.gru(embedded, hidden)
#         outputs = (outputs[:, :, :self.hidden_size] + outputs[:, :, self.hidden_size:])
#         return outputs, hidden

In [234]:
# class Attention(nn.Module):
#     def __init__(self, input_size, embed_size, hidden_size):
#         super(Attention, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.embed_size = embed_size
#         self.embed = nn.Embedding(input_size, embed_size)
#         self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#         self.v = nn.Parameter(torch.rand(hidden_size))
#         stdv = 1. / math.sqrt(self.v.size(0))
#         self.v.data.uniform_(-stdv, stdv)

#     def forward(self, src, encoder_outputs):
#         hidden = self.embed(src)
#         return 
#         timestep = encoder_outputs.size(0)
#         h = hidden.repeat(timestep, 1, 1).transpose(0, 1)
#         encoder_outputs = encoder_outputs.transpose(0, 1)  # [B*T*H]
#         energy = torch.tanh(self.attn(torch.cat((h, encoder_outputs), dim = 2)))
#         attention = torch.sum(energy, dim=2)
#         return F.softmax(attention, dim=1)

#     def score(self, hidden, encoder_outputs):
#         # [B*T*2H]->[B*T*H]
#         energy = F.relu(self.attn(torch.cat([hidden, encoder_outputs], 2)))
#         energy = energy.transpose(1, 2)  # [B*H*T]
#         v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # [B*1*H]
#         energy = torch.bmm(v, energy)  # [B*1*T]
#         return energy.squeeze(1)  # [B*T]

In [305]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        outputs = (outputs[:, :, :self.enc_hid_dim] +
                   outputs[:, :, self.enc_hid_dim:])
        return outputs, hidden

In [306]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.attn_in = enc_hid_dim + dec_hid_dim
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]
        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))
        attention = torch.sum(energy, dim=2)
        return F.softmax(attention, dim=1)

In [323]:
class Model(nn.Module):
    def __init__(self, encoder, attention, hidden_size, out_size):
        super(Model, self).__init__()
        self.encoder = encoder
        self.attention = attention
        self.out1 = nn.Linear(hidden_size, hidden_size)
        self.out2 = nn.Linear(hidden_size, out_size)

    def forward(self, src):
        batch_size = src.size(1)
        encoder_outputs, hidden = self.encoder(src)
        attn_weights = self.attention(hidden, encoder_outputs).unsqueeze(1)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  # (B,1,N)
        context = context.transpose(0, 1) 
        out = F.relu(self.out2(context))[0]
        return F.softmax(out, dim=1)

In [324]:
enc = Encoder(vocab_sz, 50, 50, 50, 0)
attn = Attention(50, 50, 50)
model = Model(enc, attn, 50, vocab_sz)

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [325]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src)
        loss = criterion(output, trg[0])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def epoch_time(start_time: int, end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 100
for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

Epoch: 01 | Time: 0m 1s
	Train Loss: 10.150 | Train PPL: 25601.076
Epoch: 02 | Time: 0m 1s
	Train Loss: 10.150 | Train PPL: 25601.076
Epoch: 03 | Time: 0m 1s
	Train Loss: 10.150 | Train PPL: 25601.076
Epoch: 04 | Time: 0m 1s
	Train Loss: 10.150 | Train PPL: 25601.076


KeyboardInterrupt: 